<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/01-functions_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Function Calling


## https://github.com/openai/openai-python

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=<FILEID>'  -O ./.env

--2024-06-14 06:30:37--  https://docs.google.com/uc?export=download&id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5
Resolving docs.google.com (docs.google.com)... 74.125.200.138, 74.125.200.113, 74.125.200.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5&export=download [following]
--2024-06-14 06:30:37--  https://drive.usercontent.google.com/download?id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 64.233.170.132, 2404:6800:4003:c1a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|64.233.170.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175 [application/octet-stream]
Saving to: ‘./.env’

./.env              100%[===================>]     175  --.-KB/s    in 0s      

2024-06

**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [ ]:
!pip install openai==0.28  python-dotenv

In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [7]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [8]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [9]:
import openai

In [10]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [11]:
print(response)

{
  "id": "chatcmpl-9Zul6wT6rHWp1Ax1Dw2XxRSJ6kijG",
  "object": "chat.completion",
  "created": 1718346688,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


In [12]:
response_message = response["choices"][0]["message"]

In [13]:
response_message

<OpenAIObject at 0x7a3f17124a90> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}

In [14]:
response_message["content"]

In [15]:
response_message["function_call"]

<OpenAIObject at 0x7a3ef96b1f80> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
}

In [16]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

In [17]:
args = json.loads(response_message["function_call"]["arguments"])

In [18]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [20]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [21]:
print(response)

{
  "id": "chatcmpl-9Zum3koBGMtPfcED2IcBwLN04jhyX",
  "object": "chat.completion",
  "created": 1718346747,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-9ZumBxNwbJjCQDjLr5ri1T08McbO6",
  "object": "chat.completion",
  "created": 1718346755,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


In [23]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-9ZumIBg97GUacL3zKzPc5reMGtNjx",
  "object": "chat.completion",
  "created": 1718346762,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


In [24]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-9ZumPQP3eelvWNbBYilAHjkU8JkA8",
  "object": "chat.completion",
  "created": 1718346769,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Please wait a moment while I check the current weather in Boston."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 13,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


In [25]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-9ZumVxR3MQ92iL7rzmBYCc2ZHdPmb",
  "object": "chat.completion",
  "created": 1718346775,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


In [26]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-9ZumbPAoRygeKRs76GJZ4SrPlTAZI",
  "object": "chat.completion",
  "created": 1718346781,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


In [27]:
messages.append(response["choices"][0]["message"])

In [28]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [29]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [30]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-9Zumxs9yXVzXPnpX6nRTrFwFdx0mh",
  "object": "chat.completion",
  "created": 1718346803,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently 72 degrees Fahrenheit with a sunny and windy forecast."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 17,
    "total_tokens": 94
  },
  "system_fingerprint": null
}
